In [ ]:
from langchain.agents import Tool, initialize_agent, AgentType, load_tools
from dotenv import load_dotenv
from langchain.utilities import (
    WikipediaAPIWrapper,
    GoogleSearchAPIWrapper,
)
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain import OpenAI, Wikipedia

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)
from langchain.chat_models import ChatOpenAI
from langchain import LLMMathChain
from langchain.document_loaders import UnstructuredURLLoader

# from langchain.retrievers.web_research import WebResearchRetriever
import os

In [ ]:
load_dotenv()

In [ ]:
google = GoogleSearchAPIWrapper()

In [ ]:
myTopic = input("Enter your topic: ")

In [ ]:
google_webpages1 = google.results(f"site:https://en.wikipedia.org {myTopic}", 5)
google_webpages2 = google.results(myTopic, 10)

In [ ]:
print(google_webpages1)
print(google_webpages2)

In [ ]:
# %pip install pip install python_magic_bin-0.4.14-py2.py3-none-win_amd64.whl
# %pip install python-magic-bin

In [ ]:
# loop over dictionary
links = []
for i in range(len(google_webpages1)):
    links.append(google_webpages1[i]["link"])

for i in range(len(google_webpages2)):
    links.append(google_webpages2[i]["link"])

print(links)

In [ ]:
loaders = UnstructuredURLLoader(urls=links)
data = loaders.load()

In [ ]:
for i in range(len(data)):
    print(f"Page {i+1}")
    print(f"Title: {data[i].metadata}")
    print("----------------------------------------------------------")

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n", "\t", "\r", "\f", "\v", "\0", "\x0b", "\x0c"],
    chunk_size=1000,
    chunk_overlap=500,
)

In [ ]:
docs = text_splitter.split_documents(documents=data)

In [ ]:
len(docs)
docs[0].metadata

In [ ]:
# %pip install faiss-cpu

In [ ]:
import pickle
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import faiss

In [ ]:
embeddings = OpenAIEmbeddings()
print(embeddings)

In [ ]:
vectorStore_openAI = FAISS.from_documents(docs, embedding=embeddings)

In [ ]:
# to store the embeddings in a faiss index
with open("faiss_store_openai.pkl", "wb") as f:
    pickle.dump(vectorStore_openAI, f)

In [ ]:
# to read the vectors stored
with open("faiss_store_openai.pkl", "rb") as f:
    vectorStore = pickle.load(f)

In [ ]:
vectorStore
print(type(vectorStore))

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

# from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

# from langchain.retrievers import PineconeHybridSearchRetriever
# from pinecone_text.sparse import BM25Encoder
from langchain.retrievers.web_research import WebResearchRetriever

# import ChatOpenAI
from langchain.chat_models import ChatOpenAI

In [ ]:
# llm = OpenAI()  # could be any model
llm = ChatOpenAI(temperature=0, model="gpt-4")

In [ ]:
import asyncio

# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorStore,
    llm=llm,
    search=google,
)


user_input = "How old is Mo Salah?"
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)
await asyncio.sleep(10)


def func():
    return qa_chain({"question": user_input})


try:
    result = asyncio.wait_for(func(), timeout=10000)
    print(result)
except Exception as e:
    print("errrrrrrrrrrror", e)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(
    llm,
    retriever=vectorStore.as_retriever(),
)

In [55]:
%pip install qdrant-client

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.18.1 requires protobuf<4,>=3.12, but you have protobuf 4.24.1 which is incompatible.



                                              0.0/132.5 kB ? eta -:--:--
     ---------                                30.7/132.5 kB ? eta -:--:--
     ---------------------------             92.2/132.5 kB 1.0 MB/s eta 0:00:01
     ------------------------------------ 132.5/132.5 kB 978.1 kB/s eta 0:00:00
                                              0.0/4.3 MB ? eta -:--:--
                                              0.1/4.3 MB 3.2 MB/s eta 0:00:02
     -                                        0.1/4.3 MB 1.3 MB/s eta 0:00:04
     --                                       0.2/4.3 MB 1.7 MB/s eta 0:00:03
     ---                                      0.3/4.3 MB 1.9 MB/s eta 0:00:03
     ----                                     0.5/4.3 MB 2.1 MB/s eta 0:00:02
     -----                                    0.6/4.3 MB 2.1 MB/s eta 0:00:02
     ------                                   0.7/4.3 MB 2.1 MB/s eta 0:00:02
     -------                                  0.8/4.3 MB 2.2 MB/s eta 0:00:

In [56]:
# 7isMRbRGP6Er1tw0ltVeQu22NjHyG1wNOoY2OxLUkat3Z7ThCIKcXQ
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://e90343c0-1763-4186-984d-dcee1b74c728.us-east-1-0.aws.cloud.qdrant.io:6333", 
    api_key="7isMRbRGP6Er1tw0ltVeQu22NjHyG1wNOoY2OxLUkat3Z7ThCIKcXQ",
)

In [57]:
from langchain.vectorstores import qdrant
os.environ["QDRANT_API_KEY"] = "7isMRbRGP6Er1tw0ltVeQu22NjHyG1wNOoY2OxLUkat3Z7ThCIKcXQ"
os.environ["QDRANT_HOST"] = "https://e90343c0-1763-4186-984d-dcee1b74c728.us-east-1-0.aws.cloud.qdrant.io:6333"

In [ ]:
qdrant_client